Uploading Phase:

In [1]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install scikit-learn

Calling the Neccessary Libraries:

In [2]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage import exposure
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import cv2
import glob
import os
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Finding Height and Width of an Image:

In [3]:
sample_image_path = '/content/drive/MyDrive/FACE IMAGES/1.1.pgm'

# Load the image in grayscale
img = cv2.imread('/content/drive/MyDrive/FACE IMAGES/1.1.pgm', cv2.IMREAD_GRAYSCALE)

# Print its shape
print("Image shape:", img.shape)

height = img.shape[0]
print("Height:", height)

Image shape: (80, 64)
Height: 80


Training Part:

In [4]:
# ----------------------------
# CONFIGURATION
# ----------------------------
DATASET_PATH = "/content/drive/MyDrive/FACE IMAGES"
IMAGE_EXTENSION = ".pgm"
NUM_SUBJECTS = 250
TRAIN_PER_SUBJECT = 2
PATCH_SIZE = 5
stride = 5
target_size = (60, 80)

# LBP configurations: (R, P)
LBP_CONFIGS = [(1, 16), (2, 16), (1, 8), (2, 8)]
REFERENCE_CONFIG = (2, 16)

# ----------------------------
# FUNCTION: RIU2 Mapping
# ----------------------------
def get_riu2_mapping(P):
    table = np.zeros(2 ** P, dtype=np.uint8)
    for i in range(2 ** P):
        binary = [(i >> j) & 1 for j in range(P)]
        binary.append(binary[0])
        transitions = sum(binary[j] != binary[j+1] for j in range(P))
        if transitions <= 2:
            table[i] = sum(binary[:P])
        else:
            table[i] = P + 1
    return table

# ----------------------------
# FUNCTION: Extract LBP histogram
# ----------------------------
def extract_lbp_histogram(block, P, R, riu2_map):
    lbp = local_binary_pattern(block, P, R, method='ror').astype(np.uint16)
    lbp_mapped = riu2_map[lbp]
    hist, _ = np.histogram(
        lbp_mapped.ravel(),
        bins=np.arange(0, P + 3),
        density=True
    )
    return hist

# ----------------------------
# FUNCTION: Augment Image
# ----------------------------
def augment_image(img):
    h, w = img.shape
    aug_imgs = []

    # Rotations
    for angle in [-10, -5, 5, 10]:
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
        rotated = cv2.warpAffine(img, M, (w, h))
        aug_imgs.append(rotated)

    # Horizontal Flip
    flipped = cv2.flip(img, 1)
    aug_imgs.append(flipped)

    # Gaussian Blur
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    aug_imgs.append(blurred)

    return aug_imgs

# ----------------------------
# PRE-COMPUTE MAPPINGS
# ----------------------------
mapping_dict = {P: get_riu2_mapping(P) for _, P in LBP_CONFIGS + [REFERENCE_CONFIG]}

# ----------------------------
# TRAINING PHASE (Concatenated Feature Vector)
# ----------------------------
train_lbp_features = []
train_labels = []

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(1, TRAIN_PER_SUBJECT + 1):
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        augmented_images = augment_image(resized)
        augmented_images.append(resized)  # Include original

        full_feature_vector = []

        for aug in augmented_images:
            denoised = cv2.fastNlMeansDenoising(aug, h=10)
            equalized = cv2.equalizeHist(denoised).astype(np.float64) / 255.0
            normalized = (equalized - np.mean(equalized)) / np.std(equalized)

            for y in range(0, normalized.shape[0] - PATCH_SIZE + 1, stride):
                for x in range(0, normalized.shape[1] - PATCH_SIZE + 1, stride):
                    block = normalized[y:y + PATCH_SIZE, x:x + PATCH_SIZE]
                    combined_hist = []
                    for R, P in LBP_CONFIGS:
                        hist = extract_lbp_histogram(block, P, R, mapping_dict[P])
                        combined_hist.extend(hist)
                    full_feature_vector.extend(combined_hist)

        train_lbp_features.append(full_feature_vector)
        train_labels.append(subject_id)

        print(f"✅ Processed {filename} | Augmentations: {len(augmented_images)} | Feature vector length: {len(full_feature_vector)}")

train_lbp_features = np.array(train_lbp_features, dtype=np.float32)
train_lbp_features = normalize(train_lbp_features, norm='l2')
train_labels = np.array(train_labels)

/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


✅ Processed 1.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 1.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 2.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 2.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 3.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 3.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 4.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 4.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 5.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 5.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 6.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 6.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 7.1.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 7.2.pgm | Augmentations: 7 | Feature vector length: 75264
✅ Processed 8.1.pgm 

Testing Part:

In [5]:
# ----------------------------
# TEST CONFIGURATION & FEATURE EXTRACTION (images 3 and 4, with augmentation)
# ----------------------------
TEST_PER_SUBJECT = 2
TEST_START_INDEX = 3  # Test on image 3 and 4

test_lbp_features = []
test_labels = []

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(TEST_START_INDEX, TEST_START_INDEX + TEST_PER_SUBJECT):
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Could not load test image: {filename}")
            continue

        resized = cv2.resize(img, target_size)
        augmented_images = augment_image(resized)
        augmented_images.append(resized)  # Include original (7 total)

        full_feature_vector = []

        for aug in augmented_images:
            denoised = cv2.fastNlMeansDenoising(aug, h=10)
            equalized = cv2.equalizeHist(denoised).astype(np.float64) / 255.0
            normalized = (equalized - np.mean(equalized)) / np.std(normalized)

            for y in range(0, normalized.shape[0] - PATCH_SIZE + 1, stride):
                for x in range(0, normalized.shape[1] - PATCH_SIZE + 1, stride):
                    block = normalized[y:y + PATCH_SIZE, x:x + PATCH_SIZE]

                    combined_hist = []
                    for R, P in LBP_CONFIGS:
                        hist = extract_lbp_histogram(block, P, R, mapping_dict[P])
                        combined_hist.extend(hist)
                    full_feature_vector.extend(combined_hist)

        test_lbp_features.append(full_feature_vector)
        test_labels.append(subject_id)

        print(f"🧪 Processed test image {filename} with {len(augmented_images)} images | Combined Feature vector length: {len(full_feature_vector)}")

# ----------------------------
# NORMALIZE TEST FEATURES
# ----------------------------
test_lbp_features = np.array(test_lbp_features, dtype=np.float32)
test_lbp_features = normalize(test_lbp_features, norm='l2')
test_labels = np.array(test_labels)

print("\n✅ All test features extracted")
print("🔢 Test feature matrix shape:", test_lbp_features.shape)


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


🧪 Processed test image 1.3.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 1.4.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 2.3.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 2.4.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 3.3.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 3.4.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 4.3.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 4.4.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 5.3.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 5.4.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 6.3.pgm with 7 images | Combined Feature vector length: 75264
🧪 Processed test image 6.4.pgm with 7 images | Combined Feature v

Recognition and Accuracy:

In [6]:
# ----------------------------
# RECOGNITION + ACCURACY (MANHATTAN DISTANCE)
# ----------------------------

correct_matches = 0
total_tests = len(test_labels)

print("\n🔍 Starting recognition using Manhattan distance...")

for idx, test_feat in enumerate(test_lbp_features):
    # Compute Manhattan (L1) distance between test feature and all training features
    distances = np.sum(np.abs(train_lbp_features - test_feat), axis=1)

    # Identify the best match (smallest distance)
    best_match_idx = np.argmin(distances)
    predicted_label = train_labels[best_match_idx]
    true_label = test_labels[idx]

    # Report the prediction
    print(f"🔍 Test #{idx + 1} | True Label: {true_label} | Predicted Label: {predicted_label}")

    # Check correctness
    if predicted_label == true_label:
        correct_matches += 1

# Compute and display final accuracy
accuracy = (correct_matches / total_tests) * 100
print(f"\n✅ Final Recognition Accuracy (Manhattan Distance): {accuracy:.2f}% ({correct_matches}/{total_tests})")



🔍 Starting recognition using Manhattan distance...
🔍 Test #1 | True Label: 1 | Predicted Label: 1
🔍 Test #2 | True Label: 1 | Predicted Label: 1
🔍 Test #3 | True Label: 2 | Predicted Label: 2
🔍 Test #4 | True Label: 2 | Predicted Label: 2
🔍 Test #5 | True Label: 3 | Predicted Label: 3
🔍 Test #6 | True Label: 3 | Predicted Label: 3
🔍 Test #7 | True Label: 4 | Predicted Label: 4
🔍 Test #8 | True Label: 4 | Predicted Label: 4
🔍 Test #9 | True Label: 5 | Predicted Label: 5
🔍 Test #10 | True Label: 5 | Predicted Label: 5
🔍 Test #11 | True Label: 6 | Predicted Label: 6
🔍 Test #12 | True Label: 6 | Predicted Label: 6
🔍 Test #13 | True Label: 7 | Predicted Label: 7
🔍 Test #14 | True Label: 7 | Predicted Label: 7
🔍 Test #15 | True Label: 8 | Predicted Label: 8
🔍 Test #16 | True Label: 8 | Predicted Label: 8
🔍 Test #17 | True Label: 9 | Predicted Label: 9
🔍 Test #18 | True Label: 9 | Predicted Label: 9
🔍 Test #19 | True Label: 10 | Predicted Label: 10
🔍 Test #20 | True Label: 10 | Predicted Lab